In [13]:
import pandas as pd

In [14]:
df = pd.read_excel('insurance_claims_raw.xlsx')
print(df.shape)
df.head()

(1000, 40)


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48.0,521585,2014-10-17,OH,250/500,1000.0,1406.91,0,466132,...,YES,71610.0,6510.0,13020.0,52080,Saab,92x,2004,Y,NaN
1,228,42.0,342868,2006-06-27,IN,250/500,2000.0,1197.22,5000000,468176,...,?,5070.0,780.0,780.0,3510,Mercedes,E400,2007,Y,NaN
2,134,29.0,687698,2000-09-06,OH,100/300,2000.0,1413.14,5000000,430632,...,NO,34650.0,7700.0,3850.0,23100,Dodge,RAM,2007,N,NaN
3,256,41.0,227811,1990-05-25,IL,250/500,2000.0,1415.74,6000000,608117,...,NO,63400.0,6340.0,6340.0,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44.0,367455,2014-06-06,IL,500/1000,1000.0,1583.91,6000000,610706,...,NO,6500.0,1300.0,650.0,4550,Accura,RSX,2009,N,NaN


In [15]:
# Count the number of missing values in each column
missing_values_count = df.isnull().sum()

# Sort the counts in descending order
missing_values_count_sorted = missing_values_count[missing_values_count > 0].sort_values(ascending=False)

# Print the result
print(missing_values_count_sorted)


_c39                       1000
authorities_contacted        93
total_claim_amount            4
policy_annual_premium         3
age                           2
policy_deductable             2
insured_hobbies               2
capital-gains                 2
incident_state                2
insured_education_level       1
injury_claim                  1
property_claim                1
dtype: int64


In [16]:
df = df.drop('_c39', axis=1)

In [17]:
# Identify categorical columns
categorical_columns = df.select_dtypes(include='object').columns

# Replace missing values with 'None' for categorical columns
df[categorical_columns] = df[categorical_columns].fillna('None')
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48.0,521585,2014-10-17,OH,250/500,1000.0,1406.91,0,466132,...,2,YES,71610.0,6510.0,13020.0,52080,Saab,92x,2004,Y
1,228,42.0,342868,2006-06-27,IN,250/500,2000.0,1197.22,5000000,468176,...,0,?,5070.0,780.0,780.0,3510,Mercedes,E400,2007,Y
2,134,29.0,687698,2000-09-06,OH,100/300,2000.0,1413.14,5000000,430632,...,3,NO,34650.0,7700.0,3850.0,23100,Dodge,RAM,2007,N
3,256,41.0,227811,1990-05-25,IL,250/500,2000.0,1415.74,6000000,608117,...,2,NO,63400.0,6340.0,6340.0,50720,Chevrolet,Tahoe,2014,Y
4,228,44.0,367455,2014-06-06,IL,500/1000,1000.0,1583.91,6000000,610706,...,1,NO,6500.0,1300.0,650.0,4550,Accura,RSX,2009,N


In [21]:
# Select numeric columns
numeric_columns = df.select_dtypes(include='number').columns

# Fill missing values with the mean of numeric columns
df_filled = df.fillna(df[numeric_columns].mean())


In [24]:
from scipy import stats

threshold = 3

numeric_columns = df_filled.select_dtypes(include='number').columns
z_scores = stats.zscore(df[numeric_columns])
outlier_indices = (abs(z_scores) > threshold).any(axis=1)
df_without_outliers = df_filled[~outlier_indices]

df_without_outliers.shape

(984, 39)

In [25]:
df_without_outliers.describe()

,months_as_customer,age,policy_number,policy_bind_date,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_date,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year
count,984.00000,984.000000,984.000000,984,984.000000,984.000000,9.840000e+02,984.000000,984.000000,984.000000,984,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000
mean,204.10874,38.955209,545952.560976,2002-02-02 00:51:13.170731648,1133.911115,1256.582839,9.878049e+05,500927.030488,25337.147262,-26714.634146,2015-01-30 08:10:14.634146304,11.667683,1.833333,0.995935,1.486789,52726.077113,7433.633074,7399.307549,37888.109756,2005.140244
min,0.00000,19.000000,100804.000000,1990-01-08 00:00:00,500.000000,433.330000,-1.000000e+06,430104.000000,0.000000,-93600.000000,2015-01-01 00:00:00,0.000000,1.000000,0.000000,0.000000,100.000000,0.000000,0.000000,70.000000,1995.000000
25%,115.75000,32.000000,335522.250000,1995-09-03 18:00:00,500.000000,1087.875000,0.000000e+00,448458.500000,0.000000,-51425.000000,2015-01-15 00:00:00,6.000000,1.000000,0.000000,0.750000,41812.500000,4277.500000,4480.000000,30220.000000,2000.000000
50%,199.50000,38.000000,533135.000000,2002-03-16 12:00:00,1000.000000,1257.020557,0.000000e+00,466445.500000,0.000000,-23250.000000,2015-01-31 00:00:00,12.000000,1.000000,1.000000,1.000000,57790.000000,6775.000000,6750.000000,42080.000000,2005.000000
75%,277.25000,44.000000,760309.250000,2008-04-21 12:00:00,2000.000000,1415.825000,0.000000e+00,603208.250000,51100.000000,0.000000,2015-02-15 00:00:00,17.000000,3.000000,2.000000,2.000000,70425.000000,11285.000000,10865.000000,50750.000000,2010.000000
max,479.00000,64.000000,999435.000000,2015-02-22 00:00:00,2000.000000,2047.590000,7.000000e+06,620962.000000,100500.000000,0.000000,2015-03-01 00:00:00,23.000000,4.000000,2.000000,3.000000,114920.000000,21450.000000,23670.000000,79560.000000,2015.000000
std,115.51018,9.169259,257678.186662,NaN,609.471516,245.114820,2.117565e+06,71505.100886,27863.850661,27972.797956,NaN,6.968344,1.017980,0.818146,1.113054,26319.898645,4886.980371,4818.346817,18857.798943,6.014111


In [26]:
df.to_csv('insurance_claims_cleaned.csv', index=False)